In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pyxlsb import open_workbook as open_xlsb
from collections import Counter
import tkinter as tk
import datetime as dt
from tkinter import filedialog, messagebox
from fpdf import FPDF

In [2]:
class PortfolioAnalysis:
    def __init__(self, file_path, initial_investment=100000, name="Nasser BOINA", risk_profile="Prudent", start_date="2019-12-01", end_date="2019-12-31"):
        self.file_path = file_path
        self.initial_investment = initial_investment
        self.name = name
        self.risk_profile = risk_profile
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.portfolio = self.load_portfolio()
        self.df_filtre = self.process_cac40_data()
        self.weights = self.calculate_weights()

    def load_portfolio(self):
        """Charge le portefeuille à partir du fichier Excel."""
        data = []
        with open_xlsb(self.file_path) as wb:
            with wb.get_sheet(1) as sheet:
                for row in sheet.rows():
                    data.append([item.v for item in row])
        
        portfolio_df = pd.DataFrame(data[1:], columns=data[0])
        portfolio_list = portfolio_df.iloc[:, 1].tolist()
        return portfolio_list

    def process_cac40_data(self):
        """Traite les données du CAC40 à partir du fichier Excel."""
        data = []
        with open_xlsb(self.file_path) as wb:
            with wb.get_sheet(3) as sheet:
                for row in sheet.rows():
                    data.append([item.v for item in row])
        
        cac40_df = pd.DataFrame(data[5:], columns=data[3])
        date_reference = datetime(1899, 12, 30)
        cac40_df[cac40_df.columns[0]] = pd.to_datetime(cac40_df[cac40_df.columns[0]].apply(lambda x: date_reference + timedelta(days=int(x))))
        cac40_df.set_index(cac40_df.columns[0], inplace=True)
        
        filtered_data = cac40_df.loc[self.start_date:self.end_date]
        columns = filtered_data.columns.to_list()
        selected_columns = [x for x in self.portfolio if x in columns]
        df_filtre = filtered_data[selected_columns]
        return df_filtre

    def selected_assets(self):
        """Sélectionne les actifs en fonction du profil de risque."""
        if self.risk_profile == "Prudent":
            return [
                "VEOLIA ENVIRON", "ENGIE", "DANONE", "SANOFI", "TOTALENERGIES",  "L'OREAL", "LVMH",
                "HERMES INTL.", "AXA",  "SCHNEIDER ELECTRIC", "LEGRAND",  "STMICROELECTRONICS", "VINCI",
                "L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.", "ESSILORLUXOTTICA", "ORANGE", "KERING",
                "BNP PARIBAS", "CREDIT AGRICOLE", "VIVENDI"
            ]
        elif self.risk_profile == "Équilibré":
            return [
                "ACCOR", "CARREFOUR", "BOUYGUES", "ARCELORMITTAL", "THALES", "SAINT GOBAIN", "PUBLICIS GROUPE", 
                "SODEXO", "PEUGEOT DEAD - CONT SEE.99278Y",  "CMPG.DES ETS.MICH.", "CAPGEMINI", "DASSAULT SYSTEMES", "SAFRAN", 
                "UNIBAIL RODAMCO WE STAPLED UNITS", "SOCIETE GENERALE", "PERNOD-RICARD", "L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.",
                "LAGARDERE GROUPE","AIRBUS", "KERING"
            ]
        else:  # Dynamique
            return [
                "ATOS", "RENAULT", "SODEXO", "UNIBAIL RODAMCO WE STAPLED UNITS", "SOCIETE GENERALE", 
                "PERNOD-RICARD", "L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.", "CREDIT AGRICOLE", 
                "PUBLICIS GROUPE", "BNP PARIBAS", "SAINT GOBAIN", "ENGIE", "PUBLICIS GROUPE", "BOUYGUES", 
                "ARCELORMITTAL", "HERMES INTL.", "AIRBUS", "SANOFI", "L'OREAL", "LVMH"
            ]

    def calculate_weights(self):
        """Calcule les poids des actifs dans le portefeuille."""
        initial_investment = self.initial_investment
        selected_assets = self.selected_assets()
        df_filtre_sampled = self.df_filtre[selected_assets]
        
        initial_prices = df_filtre_sampled.loc[self.start_date]
        weights = np.random.rand(len(initial_prices))
        weights /= np.sum(weights)
        values_invested = initial_investment * weights
        assert np.isclose(np.sum(values_invested), initial_investment), "L'investissement initial n'est pas respecté"
        
        shares_purchased = values_invested / initial_prices
        final_prices = df_filtre_sampled.loc[self.end_date]
        values_at_date_2 = shares_purchased * final_prices
        total_value_at_date_2 = np.sum(values_at_date_2)
        new_weights_at_date_2 = values_at_date_2 / total_value_at_date_2
        
        return new_weights_at_date_2

    def simulate_portfolio(self):
        """Retourne la répartition des actifs et leurs poids à la "end_date"."""
        initial_investment = self.initial_investment
        df_filtre_sampled = self.df_filtre[list(self.calculate_weights().index)]
        initial_prices = df_filtre_sampled.loc[self.start_date]
        weights = self.calculate_weights()
        values_invested = initial_investment * weights
        shares_purchased = values_invested / initial_prices
        final_prices = df_filtre_sampled.loc[self.end_date]
        values_at_date_2 = shares_purchased * final_prices
        total_value_at_date_2 = np.sum(values_at_date_2)
        new_weights_at_date_2 = values_at_date_2 / total_value_at_date_2
        
        return df_filtre_sampled.columns.tolist(), new_weights_at_date_2

    def plot_pie_chart(self, labels, weights, filename='pie_chart.png'):
        """Génère un graphique en camembert pour les poids des actifs."""
        explode = [0.05] * len(labels)
        fig1, ax1 = plt.subplots(figsize=(12, 8))
        wedges, texts, autotexts = ax1.pie(weights, explode=explode, labels=labels, autopct='%.2f%%', startangle=140)
        
        for text in texts + autotexts:
            text.set_fontsize(12)
        
        ax1.legend(wedges, labels, title="Actifs", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        plt.title(f"Répartition du Portefeuille au {self.end_date}")
        plt.savefig(filename, bbox_inches='tight')
        plt.close(fig1)

    @staticmethod
    def compute_sharpe_ratio(k, avg_return, risk_free_rate, std_return):
        """Calcule le ratio de Sharpe."""
        return k * (avg_return - risk_free_rate) / std_return

    @staticmethod
    def normalize_data(df):
        """Normalise les données."""
        return df / df.iloc[0, :]

    @staticmethod
    def compute_daily_returns(df):
        """Calcule les rendements quotidiens."""
        daily_returns = df.pct_change()
        daily_returns.iloc[0, :] = 0
        return daily_returns

    def assess_portfolio(self, risk_free_rate=0.00008, sample_freq=252.0):
        """Évalue le portefeuille."""
        prices = self.df_filtre
        port_val = self.get_portfolio_value(prices, self.weights, self.initial_investment)
        cr, adr, sddr, sr = self.get_portfolio_stats(port_val, risk_free_rate, sample_freq)
        ev = port_val.iloc[-1, 0]
        return cr, adr, sddr, sr, ev

    def get_portfolio_value(self, prices, allocs, sv):
        """Calcule la valeur du portefeuille."""
        norm_prices = self.normalize_data(prices)
        
        if norm_prices.index.duplicated().any():
            norm_prices = norm_prices[~norm_prices.index.duplicated(keep='first')]
        
        if allocs.index.duplicated().any():
            allocs = allocs[~allocs.index.duplicated(keep='first')]
        
        alloc_prices = norm_prices * allocs
        pos_vals = alloc_prices * sv
        port_val = pos_vals.sum(axis=1).to_frame()
        port_val.columns = ["port_val"]
        
        return port_val

    def get_portfolio_stats(self, port_val, daily_rf, samples_per_year):
        """Calcule les statistiques du portefeuille."""
        cr = port_val.iloc[-1, 0] / port_val.iloc[0, 0] - 1
        daily_returns = self.compute_daily_returns(port_val)[1:]
        adr = daily_returns["port_val"].mean()
        sddr = daily_returns["port_val"].std()
        sr = self.compute_sharpe_ratio(np.sqrt(samples_per_year), adr, daily_rf, sddr)
        
        return cr, adr, sddr, sr

    def plot_comparison(self, port_val, prices_CAC40, filename='comparison.png'):
        """Génère un graphique de comparaison des rendements cumulatifs."""
        cumulative_returns_portfolio = (1 + self.compute_daily_returns(port_val)).cumprod() - 1
        cumulative_returns_CAC40 = (1 + prices_CAC40.pct_change()).cumprod() - 1
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(cumulative_returns_portfolio, label='Portfolio Returns', color='blue')
        ax.plot(cumulative_returns_CAC40, label='CAC40 Returns', color='red')
        ax.set_title('Cumulative Returns Comparison')
        ax.set_xlabel('Date')
        ax.set_ylabel('Cumulative Returns')
        ax.legend()
        plt.tight_layout()
        plt.savefig(filename)
        plt.close(fig)

    def plot_top(self, returns, filename='top_performers.png'):
        """Génère un graphique des meilleures performances."""
        top = returns.nlargest(5) - 1
        plt.figure(figsize=(10, 4))
        ax = top.plot(kind='bar', color='green')
        ax.set_xticklabels(top.index, rotation=0)
        plt.title('Top Performers')
        plt.ylabel('Cumulative Returns')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()

    def plot_flops(self, returns, filename='flop_performers.png'):
        """Génère un graphique des pires performances."""
        flops = returns.nsmallest(5) - 1 
        plt.figure(figsize=(10, 4))
        ax = flops.plot(kind='bar', color='red')
        ax.set_xticklabels(flops.index, rotation=0)
        plt.title('Flop Performers')
        plt.ylabel('Cumulative Returns')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()

    def plot_sector_distribution(self, sector_data, filename='sector_distribution.png'):
        """Génère un graphique de distribution des secteurs."""
        sectors = list(sector_data.values())
        sector_counts = Counter(sectors)
        labels = list(sector_counts.keys())
        sizes = list(sector_counts.values())
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.bar(labels, sizes, color='blue')
        plt.title("Répartition des secteurs d'activité des entreprises du portefeuille")
        plt.xlabel("Secteurs d'activité")
        plt.ylabel("Nombre d'entreprises")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close(fig)

    def get_sector_data(self, entreprises_cac40):
        """Récupère les données sectorielles pour les entreprises sélectionnées."""
        selected_assets = self.selected_assets()
        secteurs_activite = {entreprise: entreprises_cac40[entreprise] for entreprise in selected_assets}
        return secteurs_activite


In [ ]:
class PortfolioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Analyse de Portefeuille")
        
        # Interface de sélection du fichier
        self.label_file = tk.Label(root, text="1. Choisir le fichier CAC40_closing_94_to_22.xlsb:")
        self.label_file.pack(pady=10)
        
        self.button_file = tk.Button(root, text="Sélectionner Fichier", command=self.load_file)
        self.button_file.pack(pady=10)
        
        # Interface pour entrer le nom
        self.label_name = tk.Label(root, text="2. Entrez votre nom et prénom:")
        self.label_name.pack(pady=10)
        
        self.entry_name = tk.Entry(root)
        self.entry_name.pack()
        
        # Interface pour entrer le montant de l'investissement initial
        self.label_investment = tk.Label(root, text="3. Entrez le montant de l'investissement initial:")
        self.label_investment.pack(pady=10)
        
        self.entry_investment = tk.Entry(root)
        self.entry_investment.pack()

        # Interface pour sélectionner le profil de risque
        self.label_risk_profile = tk.Label(root, text="4. Quel est votre profil de risque?")
        self.label_risk_profile.pack(pady=10)

        self.risk_profile = tk.StringVar(value="Prudent")
        self.radio_prudent = tk.Radiobutton(root, text="Prudent", variable=self.risk_profile, value="Prudent")
        self.radio_prudent.pack()
        self.radio_equilibre = tk.Radiobutton(root, text="Équilibré", variable=self.risk_profile, value="Équilibré")
        self.radio_equilibre.pack()
        self.radio_dynamique = tk.Radiobutton(root, text="Dynamique", variable=self.risk_profile, value="Dynamique")
        self.radio_dynamique.pack()
        
        # Interface pour entrer les dates de début et de fin du reporting
        self.label_start_date = tk.Label(root, text="5. Entrez la date de début du reporting (YYYY-MM-DD):")
        self.label_start_date.pack(pady=10)
        
        self.entry_start_date = tk.Entry(root)
        self.entry_start_date.pack()
        self.entry_start_date.insert(0, "2019-10-01")
        
        self.label_end_date = tk.Label(root, text="6. Entrez la date de fin du reporting (YYYY-MM-DD):")
        self.label_end_date.pack(pady=10)
        
        self.entry_end_date = tk.Entry(root)
        self.entry_end_date.pack()
        self.entry_end_date.insert(0, "2019-12-31")

        # Boutons pour lancer l'analyse et créer le reporting
        self.run_button = tk.Button(root, text="7. Lancer Analyse", command=self.run_analysis, state=tk.DISABLED)
        self.run_button.pack(pady=10)
        
        self.report_button = tk.Button(root, text="9. Créer un reporting", command=self.create_report, state=tk.DISABLED)
        self.report_button.pack(pady=10)
        
        self.file_path = None
        self.initial_investment = None

    def load_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Fichiers Excel", "*.xlsb")])
        if self.file_path:
            messagebox.showinfo("Fichier Sélectionné", f"Fichier sélectionné: {self.file_path}")
            self.run_button.config(state=tk.NORMAL)

    def run_analysis(self):
        if self.file_path:
            try:
                self.initial_investment = float(self.entry_investment.get())
                self.name = str(self.entry_name.get())
                self.risk_profile_value = self.risk_profile.get()
                start_date = self.entry_start_date.get()
                end_date = self.entry_end_date.get()
                
                # Validation des dates
                start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
                end_date = dt.datetime.strptime(end_date, "%Y-%m-%d")

                self.portfolio_analysis = PortfolioAnalysis(self.file_path, self.initial_investment, self.name, self.risk_profile_value, start_date, end_date)
                self.portfolio, self.weights = self.portfolio_analysis.simulate_portfolio()
                self.portfolio_analysis.plot_pie_chart(self.portfolio, self.weights, filename='pie_chart.png')

                cumulative_returns = (1 + self.portfolio_analysis.compute_daily_returns(self.portfolio_analysis.df_filtre)).cumprod()
                self.portfolio_analysis.plot_top(cumulative_returns.loc[end_date], filename='top_performers.png')
                self.portfolio_analysis.plot_flops(cumulative_returns.loc[end_date], filename='flop_performers.png')

                entreprises_cac40 = {
                    "LAGARDERE GROUPE": "Médias et divertissement",
                    "ATOS": "Services informatiques et logiciels",
                    "PUBLICIS GROUPE": "Communication",
                    "ACCOR": "Hôtellerie et tourisme",
                    "CARREFOUR": "Distribution",
                    "RENAULT": "Automobile",
                    "VEOLIA ENVIRON": "Services aux collectivités",
                    "BOUYGUES": "Construction et Télécommunications",
                    "SODEXO": "Services aux entreprises",
                    "ARCELORMITTAL": "Métaux et Mines",
                    "CAPGEMINI": "Services informatiques et logiciels",
                    "THALES": "Aérospatiale et Défense",
                    "PEUGEOT DEAD - CONT SEE.99278Y": "Automobile",
                    "CMPG.DES ETS.MICH.": "Pneumatiques",
                    "LEGRAND": "Équipements électriques",
                    "UNIBAIL RODAMCO WE STAPLED UNITS": "Immobilier commercial",
                    "SAINT GOBAIN": "Matériaux de construction",
                    "STMICROELECTRONICS": "Technologies et semi-conducteurs",
                    "SOCIETE GENERALE": "Services financiers",
                    "VIVENDI": "Médias et divertissement",
                    "ENGIE": "Énergie",
                    "ORANGE": "Télécommunications",
                    "CREDIT AGRICOLE": "Services financiers",
                    "DASSAULT SYSTEMES": "Services informatiques et logiciels",
                    "PERNOD-RICARD": "Boissons",
                    "DANONE": "Agroalimentaire",
                    "SCHNEIDER ELECTRIC": "Équipements électriques",
                    "L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.": "Aérospatiale et Défense",
                    "VINCI": "Construction",
                    "ESSILORLUXOTTICA": "Santé",
                    "AXA": "Assurances",
                    "SAFRAN": "Aérospatiale et Défense",
                    "BNP PARIBAS": "Services financiers",
                    "KERING": "Luxe",
                    "HERMES INTL.": "Luxe",
                    "AIRBUS": "Aérospatiale et Défense",
                    "SANOFI": "Santé",
                    "TOTALENERGIES": "Énergie",
                    "L'OREAL": "Beauté",
                    "LVMH": "Luxe"
                }

                secteur_data = self.portfolio_analysis.get_sector_data(entreprises_cac40)
                self.portfolio_analysis.plot_sector_distribution(secteur_data, filename='sector_distribution.png')

                self.cr, self.adr, self.sddr, self.sr, self.ev = self.portfolio_analysis.assess_portfolio()
                
                df_cac40 = pd.read_csv("CAC40_Index.csv", sep=";")
                df_cac40[df_cac40.columns[0]] = pd.to_datetime(df_cac40[df_cac40.columns[0]], format='%d/%m/%Y')
                df_cac40.set_index(df_cac40.columns[0], inplace=True)
                prices_CAC40 = df_cac40.loc[start_date:end_date]['Close']

                port_val = self.portfolio_analysis.get_portfolio_value(self.portfolio_analysis.df_filtre, self.portfolio_analysis.weights, self.initial_investment)
                self.portfolio_analysis.plot_comparison(port_val, prices_CAC40, filename='comparison.png')
                
                messagebox.showinfo("Analyse Terminée", "L'analyse du portefeuille est terminée. Les graphiques ont été générés.")
                self.report_button.config(state=tk.NORMAL)
            except Exception as e:
                messagebox.showerror("Erreur", f"Une erreur s'est produite: {e}")

    def create_report(self):
        try:
            pdf = FPDF()

            pdf.add_page()
            pdf.set_font("Arial", "B", 16)
            pdf.set_margins(5, 5, 5)
            w, h = 210, 297
            pdf.set_text_color(0, 0, 255)
            pdf.cell(0, 10, "Private Banking of the Vexin", 0, 1, 'C')
            pdf.ln(10)
            
            # Afficher le nom et le profil de risque en haut à droite
            pdf.set_xy(w - 50, 15)
            pdf.set_text_color(0, 0, 0)
            pdf.set_font("Arial", "", 10)
            pdf.cell(40, 10, f"Nom du Client: {self.name}", 0, 1, 'R')

            pdf.set_xy(w - 50, 25)
            pdf.cell(40, 10, f"Profil de Risque: {self.risk_profile_value}", 0, 1, 'R')
            
            # En-têtes du tableau
            pdf.set_fill_color(51, 102, 255)
            pdf.set_draw_color(0, 0, 0)
            pdf.set_font("Arial", "B", 12)
            pdf.set_text_color(255, 255, 255)
            pdf.cell(40, 7, "Final Value", border=1, ln=0, align='C', fill=True)
            pdf.cell(40, 7, "Average Daily return", border=1, ln=0, align='C', fill=True)
            pdf.cell(40, 7, "Sd of daily return", border=1, ln=0, align='C', fill=True)
            pdf.cell(40, 7, "Sharpe Ratio", border=1, ln=0, align='C', fill=True)
            pdf.cell(40, 7, "Cumulative Return", border=1, ln=1, align='C', fill=True)
            
            # Valeurs des résultats
            pdf.set_text_color(0, 0, 0)
            pdf.cell(40, 7, f"{self.ev:.4f}", border=1, ln=0, align='C')
            pdf.cell(40, 7, f"{self.adr:.4f}", border=1, ln=0, align='C')
            pdf.cell(40, 7, f"{self.sddr:.2f}", border=1, ln=0, align='C')
            pdf.cell(40, 7, f"{self.sr:.2f}", border=1, ln=0, align='C')
            pdf.cell(40, 7, f"{self.cr:.2f}", border=1, ln=1, align='C')

            # Ajouter les graphiques
            image_width, image_height = 160, 80
            x_pos = (w - image_width) / 2
            y_pos = pdf.get_y() + 10

            pdf.image("pie_chart.png", x=x_pos, y=y_pos, w=image_width, h=image_height)
            y_pos += image_height + 10
            pdf.image("comparison.png", x=x_pos, y=y_pos, w=image_width, h=image_height)

            pdf.add_page()
            x_pos_left, x_pos_right = 10, 110
            pdf.image("top_performers.png", x=x_pos_left, y=10, w=95, h=55)
            pdf.image("flop_performers.png", x=x_pos_right, y=10, w=95, h=55)
            pdf.image("sector_distribution.png", x=23, y=70, w=160, h=100)

            # Enregistrer le PDF
            pdf.output("Portfolio_Reporting.pdf")
            messagebox.showinfo("Reporting Créé", "Le reporting PDF a été créé avec succès.")
        except Exception as e:
            messagebox.showerror("Erreur", f"Une erreur s'est produite lors de la création du reporting: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PortfolioApp(root)
    root.mainloop()
